# Objetivo

Dada uma planilha em Excel, inserir os dados da planilha num banco de dados PostgreSQL.

Dica: você pode usar Python junto com psycopg2 para se conectar no PostgreSQL.
    
Passos:

* Criar uma virtualenv

* Instalar psycopg2

* Ler os dados da planilha Excel

* Tratar os dados, se necessário
* Inserir os dados no banco PostgreSQL
* Coloque seu projeto no Gitlab, ou Github pessoal e me manda o link do repositório

# Dependências 

## Biblioteca

In [9]:
import psycopg2
import pandas as pd
from decouple import config

## Dados e Constantes

In [10]:
df = pd.read_excel('categoria.xlsx')
df_1 = pd.read_excel('produto.xlsx')
df_2 = pd.read_excel('produtos-com-categoria.xlsx')

In [11]:
#tabela categoria
df.head()

,id,categoria
0,661,bebidas
1,220,sobremesas
2,485,cereais
3,617,frios
4,801,limpeza


In [12]:
#Tabela produtos
df_1.head()

,id,produto,preco,categoria_id
0,556,customer identify,562.64,801
1,626,town one,674.67,661
2,524,hotel hard,579.58,617
3,347,tend approach,664.32,834
4,869,method paper,99.22,530


In [13]:
#Produtos com categorias
df_2.head()

,id,produto,preco,categoria
0,1,uva,2.59,670
1,2,Suco de uva,10.47,101
2,3,Suco de manga,10.47,101
3,4,Suco de maça,10.47,101
4,5,Suco de laranja,10.47,101


## Inserindo os dados no banco Psql

In [14]:
try:
    connection = psycopg2.connect(user = "my_usuario",
                                  password = config("PASSWORD"),
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "my_db")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'my_usuario', 'dbname': 'my_db', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.9 (Ubuntu 11.9-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-10ubuntu2) 9.3.0, 64-bit',) 



In [15]:
cur = connection.cursor()

## Tabela `categoria`

In [16]:
#Criando tabela para categoria
cur.execute("""CREATE TABLE IF NOT EXISTS categoria (id SERIAL PRIMARY KEY, nome VARCHAR(50));""")

In [17]:
for i,row in df.iterrows():
    name = row["categoria"]
    cur.execute(f"INSERT INTO categoria (nome) VALUES ('{name}')")

In [18]:
connection.commit()

In [ ]:
#cur.execute("DELETE FROM produtos where id > 10;")

In [19]:
cur.execute("SELECT * FROM categoria;")
cur.fetchall()

[(1, 'bebidas'),
 (2, 'sobremesas'),
 (3, 'cereais'),
 (4, 'frios'),
 (5, 'limpeza'),
 (6, 'hortifruti'),
 (7, 'perfumaria'),
 (8, 'mercearia'),
 (9, 'bazar'),
 (10, 'pet shop'),
 (11, 'bebidas'),
 (12, 'sobremesas'),
 (13, 'cereais'),
 (14, 'frios'),
 (15, 'limpeza'),
 (16, 'hortifruti'),
 (17, 'perfumaria'),
 (18, 'mercearia'),
 (19, 'bazar'),
 (20, 'pet shop')]

## Tabela `produtos`

In [27]:
#Criando tabela para produtos. 
cur.execute("""CREATE TABLE IF NOT EXISTS produtos(id SERIAL PRIMARY KEY, nome_produto VARCHAR(50), valor_produtos decimal);""")

In [28]:
for i, row in df_1.iterrows():
    nome_produto = row["produto"]
    valor_produtos = row["preco"]
    cur.execute(f"INSERT INTO produtos (nome_produto, valor_produtos) VALUES ('{nome_produto}','{valor_produtos}')")

In [29]:
connection.commit()

In [30]:
cur.execute("SELECT * FROM produtos;")
cur.fetchall()

[(1, 'customer identify', Decimal('562.64')),
 (2, 'town one', Decimal('674.67')),
 (3, 'hotel hard', Decimal('579.58')),
 (4, 'tend approach', Decimal('664.32')),
 (5, 'method paper', Decimal('99.22')),
 (6, 'take control', Decimal('98.34')),
 (7, 'local want', Decimal('239.51')),
 (8, 'necessary effort', Decimal('372.86')),
 (9, 'close no', Decimal('35.53')),
 (10, 'soldier south', Decimal('452.04')),
 (11, 'quickly trial', Decimal('463.9')),
 (12, 'site old', Decimal('149.96')),
 (13, 'central establish', Decimal('550.27')),
 (14, 'half owner', Decimal('607.89')),
 (15, 'bed detail', Decimal('501.84')),
 (16, 'newspaper government', Decimal('142.43')),
 (17, 'traditional your', Decimal('113.58')),
 (18, 'safe TV', Decimal('95.67')),
 (19, 'language firm', Decimal('133.95')),
 (20, 'go economic', Decimal('132.42')),
 (21, 'bed forget', Decimal('72.15')),
 (22, 'could point', Decimal('31.62')),
 (23, 'thus very', Decimal('180.93')),
 (24, 'different mother', Decimal('244.2')),
 (25, '

## Tabela `produtos_categoria`

In [31]:
#Produtos com categoria
cur.execute("""CREATE TABLE IF NOT EXISTS produtos_categoria (id SERIAL PRIMARY KEY, nome_produto VARCHAR(50), valor_produto decimal, categoria_tipo INT REFERENCES categoria(id));""")

In [32]:
connection.commit()

In [33]:
for i, row in df_2.iterrows():
    nome_produto = row["produto"]
    valor_produto = row["preco"]
    categoria_tipo = row["categoria"]
    # cur.execute(f"INSERT INTO produtos_categoria (nome_produto, valor_produto, categoria_tipo) VALUES ('{nome_produto}','{valor_produto}',{categoria_tipo})")

In [ ]:
#cur.execute("DELETE FROM produtos where id > 64;")
#cur.execute("DROP TABLE IF EXISTS produtos;")
#cur.execute("ALTER TABLE produtos DROP COLUMN valor_produto;")
#cur.execute("ALTER TABLE produtos ADD valor_produtos decimal;")